In [ ]:
import sys
sys.path.append('../')

import uniflocpy.uWell.deviation_survey as dev_sev
import uniflocpy.uTools.data_workflow as utool
import uniflocpy.uWell.uPipe as Pipe
import uniflocpy.uWell.Self_flow_well as self_flow_well
import plotly.graph_objs as go
import numpy as np
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import tools
import pandas as pd
init_notebook_mode(connected=True)
import scipy.interpolate as interpolate
import matplotlib.pyplot as plt
data = utool.Data()
from uniflocpy.uTools import plotly_workflow
import re
import uniflocpy.uPVT.BlackOil_model as BlackOil_model
import uniflocpy.uTemperature as uTemperature
import uniflocpy.uReservoir.IPR_simple_line as IPR_simple_line
import uniflocpy.uTools.plotly_workflow as plotly_workflow 
import uniflocpy.uValidation.python_api as python_api

In [ ]:
pb_bar = 9 * 10**5
fluid_data = {"rsb_m3m3" : 56,
"gamma_oil" : 0.86,
"gamma_gas" : 1.45/1.293}

well_data = {"h_intake_mes_m" :1211,
"h_intake_vert_m" : 1211,
"h_bottomhole_mes_m" : 1757, 
"h_bottomhole_vert_m": 1757,
             
"geothermal_grad_cm":0.02,
"t_bottomhole_c" :40,
"t_earth_init_in_reservoir_c" : 40,
'p_bottomhole_bar':155.5,
"d_casing_inner_m" : 0.133,
"d_tube_inner_m" : 0.0503,
"qliq_on_surface_m3day":240,
"fw_on_surface_perc":25,
            
't_wellhead_c':20,
'p_wellhead_bar':9.75}
real_measurements = pd.DataFrame({'p_survey_mpa': [0.975, 8.495, 9.44, 10.365, 10.902, 11.272, 12.085, 12.907, 13.785, 14.67, 15.55],
                                 'h_mes_survey_m': [0, 957, 1057, 1157, 1211, 1257, 1357, 1457, 1557, 1657, 1757]})

In [ ]:
real_measurements["p_survey_bar"] = real_measurements["p_survey_mpa"] * 10
real_measurements = real_measurements.set_index(real_measurements['h_mes_survey_m'])
real_measurements.index.name = 'well.h_calculated_mes_m'

In [ ]:
calc_options ={"step_lenth_in_calc_along_wellbore_m":24.35,
                "without_annulus_space":False}

In [ ]:
reservoir = IPR_simple_line.IPRSimpleLine()
ipr_m3daybar = reservoir.calc_pi_m3daybar(well_data['qliq_on_surface_m3day'], well_data['p_bottomhole_bar'], 250)

In [ ]:
fluid_data = {"rsb_m3m3" : 56,
"gamma_oil" : 0.86,
"gamma_gas" : 0.9}

In [ ]:
pipe = Pipe.Pipe()

In [ ]:
pipe = Pipe.Pipe(temp_cor=uTemperature.temp_cor_simple_line.SimpleLineCor())


In [ ]:
blackoil_option = BlackOil_model.BlackOil_option()

In [ ]:


blackoil_option.b_wat_cor_number = 1
blackoil_option.mu_wat_cor_number = 1
blackoil_option.rho_wat_cor_number = 1

In [ ]:
fluid = BlackOil_model.Fluid(**fluid_data, option = blackoil_option)


In [ ]:

simple_well = self_flow_well.self_flow_well(fluid=fluid, reservoir = reservoir, pipe=pipe,
                                            **well_data, **calc_options)
simple_well.well_work_time_sec = 1

In [ ]:
#simple_well.calc_all_from_down_to_up()

In [ ]:
simple_well.calc_all_from_up_to_down()

In [ ]:
result_df = simple_well.data.get_data_as_df()
result_df.to_excel('test.xlsx')
result_df = result_df.set_index('well.h_calculated_mes_m')
result_df.head()

In [ ]:
result_df = result_df.join(real_measurements, how = 'outer')

In [ ]:
group_patterns = [["p_bar", 'fl'], ['t_c', 'fl'], ['well.t_calculated_earth_init'], ['survey']]
antipatterns = ['mu', 'h_mes_survey', 'mpa']
plotly_workflow.plot_by_patterns(result_df, group_patterns, antipatterns, plot_name='Распределение свойств по глубине')

In [ ]:
simple_well.save_all = True
results = []
for i in range(1, 300, 15):

    print(i)
    simple_well.data.clear_data()
    simple_well.qliq_on_surface_m3day = i
    simple_well.calc_all_from_up_to_down()
    this_result_df = simple_well.data.get_data_as_df()
    this_result_df = this_result_df.set_index('well.h_calculated_mes_m')
    results.append(this_result_df)
    
    this_result_df = this_result_df.add_prefix(f"qliq_m3day: {i}, ")
    try:
        result_df_to_plot = result_df_to_plot.join(this_result_df)
    except:
        result_df_to_plot = this_result_df.copy()

In [ ]:
result_df_analysis = None
for i in results:
    qliq_on_surface_m3day = i['well.qliq_on_surface_m3day'][0]
    this_df = pd.DataFrame({'well.p_bottomhole_bar': i['well.p_bottomhole_bar'][0]}, index = [qliq_on_surface_m3day])
    try:
        result_df_analysis = result_df_analysis.append(this_df)
    except:
        result_df_analysis = this_df.copy()

In [ ]:
all_traces = plotly_workflow.create_traces_list_for_all_columms(result_df_analysis, 'lines+markers')
plotly_workflow.plot_func(all_traces, 'Распределение свойств по глубине', 'check.html', iplot_option = True)


In [ ]:
simple_well.save_all = False
result_df_analysis = None
for j in range(1, 99, 20):
    simple_well.fw_on_surface_perc = j
    results_q = []
    results_p = []
    for i in range(1, 300, 40):
        simple_well.p_wellhead_bar = 20
        simple_well.data.clear_data()
        simple_well.qliq_on_surface_m3day = i
        simple_well.calc_all_from_up_to_down()
        results_q.append(i)
        results_p.append(simple_well.p_bottomhole_bar)
    one_df = pd.DataFrame({f"p_bottomhole_bar {j}": results_p}, index = results_q)

    try:
        result_df_analysis = result_df_analysis.append(one_df)
    except:
        result_df_analysis = one_df.copy()

In [ ]:
all_traces = plotly_workflow.create_traces_list_for_all_columms(result_df_analysis, 'lines+markers')
plotly_workflow.plot_func(all_traces, 'Распределение свойств по глубине', 'check.html', iplot_option = True)


In [ ]:
simple_well.save_all = False
result_df_analysis = None
for j in range(1, 500, 50):
    simple_well.fw_on_surface_perc = 20
    simple_well.pipe.fluid_flow.fl.rsb_m3m3 = j
    results_q = []
    results_p = []
    for i in range(1, 300, 40):
        simple_well.p_wellhead_bar = 20
        simple_well.data.clear_data()
        simple_well.qliq_on_surface_m3day = i
        simple_well.calc_all_from_up_to_down()
        results_q.append(i)
        results_p.append(simple_well.p_bottomhole_bar)
    one_df = pd.DataFrame({f"p_bottomhole_bar {j}": results_p}, index = results_q)

    try:
        result_df_analysis = result_df_analysis.append(one_df)
    except:
        result_df_analysis = one_df.copy()

In [ ]:
all_traces = plotly_workflow.create_traces_list_for_all_columms(result_df_analysis, 'lines+markers')
plotly_workflow.plot_func(all_traces, 'Распределение свойств по глубине', 'check.html', iplot_option = True)


In [ ]:
amount_of_threads = 6
from multiprocessing import Pool

In [ ]:
import uniflocpy.uWell.mp_test as mp_test

In [ ]:
if __name__ == '__main__':
    with Pool(6) as p:
        result = p.map(mp_test.calc_well_with_one_parameter, [1, 20, 50, 200, 500, 1000])

In [ ]:
result_df  = None
for i in result:
    try:
        result_df = result_df.join(i)
    except:
        result_df = i.copy()
    

In [ ]:
result_df

In [ ]:
all_traces = plotly_workflow.create_traces_list_for_all_columms(result_df, 'lines+markers')
plotly_workflow.plot_func(all_traces, 'Распределение свойств по глубине', 'check.html', iplot_option = True)
